In [174]:
from config import sparqlTerms, mig_ns, sparql_mig_test, sparql_mig_simple, sparql_mig_dev
from SPARQLWrapper import JSON, SPARQLWrapper
import re, os, json, requests
import nltk
from nltk import word_tokenize

In [196]:
class Query:
    def __init__(self, strlen, delim):
        self.len = strlen
        self.delim = delim
        self.sparql = SPARQLWrapper(sparql_mig_dev)
        

    def getTypebyLen(self):
        self.sparql.setMethod("POST")
        self.sparql.setReturnFormat(JSON)
        self.out = []
        query = "prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT ?type (count(?type) as ?count) (sample(?s) as ?resource) where {?s dcterm:type ?type {select ?s ?value WHERE {?s dcterm:created ?value. filter(contains(str(?value), str('%s')) && STRLEN(str(?value)) = %s)}}} Group by ?type" % (self.delim, self.len)
        self.sparql.setQuery(query)
        r  = self.sparql.query().convert()['results']['bindings']
        for triple in r:
            if triple['type']['value'] != '':
                self.out.append(triple['type']['value'] + " \t " + triple['count']['value'] + "\t" + triple['resource']['value'] + "\n") 
                
    def getDateLen(self):
        self.sparql.setMethod("POST")
        self.sparql.setReturnFormat(JSON)
        self.out = []
        query = "prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT ?value (count(?value) as ?count) (sample(?s) as ?resource) WHERE {?s dcterm:created ?value. filter(contains(str(?value), str('%s')) && (STRLEN(str(?value)) = %s))} GROUP BY ?value" % (self.delim, self.len)
        self.sparql.setQuery(query)
        r  = self.sparql.query().convert()['results']['bindings']
        for triple in r:
            if triple['value']['value'] != '':
                self.out.append(triple['value']['value'] + " \t " + triple['count']['value'] + "\t" + triple['resource']['value'] + "\n")    
                
    def SampleDateLen(self):
        self.sparql.setMethod("POST")
        self.sparql.setReturnFormat(JSON)
        self.out = []
        query = "prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT ?value ?resource WHERE {?resource dcterm:created ?value. filter(contains(str(?value), str('%s')) && (STRLEN(str(?value)) = %s))} LIMIT 1" % (self.delim, self.len)
        self.sparql.setQuery(query)
        r  = self.sparql.query().convert()['results']['bindings']
        for triple in r:
            if triple['value']['value'] != '':
                self.out.append(triple['value']['value'] + " \t " + triple['resource']['value'] + "\n")  
                
    def isVersionOf(self):
        self.sparql.setMethod("POST")
        self.sparql.setReturnFormat(JSON)
        query = "prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT ?s ?value WHERE {?s dcterm:isVersionOf ?value. {select ?s where {?s dcterm:created ?v filter(?v='')}} filter(?value != '')}"
        self.sparql.setQuery(query)
        r  = self.sparql.query().convert()['results']['bindings']
        for triple in r:
            text = triple['value']['value']
            resource = triple['s']['value']
            print (resource)
            tokens = word_tokenize(text)
            for n,i in enumerate(tokens):
                if i == "," :
                    del tokens[n]
                if i == ")" :
                    del tokens[n]
                if i == "(" :
                    del tokens[n]
            years = Word(tokens)
            yearday = years.getyday()
            if yearday == None:
                # if a yyyy/mm/dd was not found search for other types of dates
                yearmonth = years.getymonth()
                if yearmonth == None:
                    # if a yyyy/mm was not found search for other types of dates
                    trans = years.getyear()
                    yyyy = Search(trans, resource)
                    yyyy.con()                    

In [161]:
def main():
    for i in range(1, 31):
        for j in [";", ":", "-", "/", ".", ",", " "]:
            dat = Query(i, j)
            with open("Dates/len " + str(i) + " dates.tsv" , "a") as file:
                print ("len is: " + str(i), "Delimiter is: " + str(j))
                dd = dat.getDateLen()
                file.writelines(j + "\n")
                file.writelines(dat.out)
                file.writelines("-----------------------------------------------------------" + "\n")
            with open("Types/len " + str(i) + " types.tsv" , "a") as file:
                print ("len is: " + str(i), "Delimiter is: " + str(j))
                dd = dat.getTypebyLen()
                file.writelines(j + "\n")
                file.writelines(dat.out)
                file.writelines("-----------------------------------------------------------" + "\n")
            with open("Sample/len " + str(i) + " sample.tsv" , "a") as file:
                print ("len is: " + str(i), "Delimiter is: " + str(j))
                dd = dat.SampleDateLen()
                file.writelines(j + "\n")
                file.writelines(dat.out)
                file.writelines("-----------------------------------------------------------" + "\n")

In [80]:
if __name__ == "__main__":
	main()

len is: 1 Delimiter is: ;


FileNotFoundError: [Errno 2] No such file or directory: 'Types/len 1 types.tsv'

In [197]:
class Word:
    def __init__(self, tokens):
        self.word = tokens
        self.out = []
        self.output = [""]
        
    def getyear(self):
        # find all years with a 4 digit format (between 1900 and 2018)
        self.year = [i for i, w in enumerate(self.word) if re.search('^\d{4}$', w)]
        #print (self.year)
        for n,year in enumerate(self.year):
            self.out.append({'year': '', "grams": {"1-gram": "", "-1-gram": "", "2-gram": "", "-2-gram": "", "3-gram": "", "-3-gram": "", "4-gram": "", "-4-gram": "", "5-gram": "", "-5-gram": ""}})
            if int(self.word[year]) > 1900 and int(self.word[year]) < 2018:
                #index = self.word.index(year)
                self.out[n]["year"] = self.word[year]
                for j in range(1, 6):
                    # preceding grams
                    if j + year < len(self.word): 
                        self.out[n]["grams"][str(j) + "-gram"] = self.word[year + j] 
                    #succeeding grams 
                    if year - j > 0:
                        self.out[n]["grams"]["-" + str(j) + "-gram"] = self.word[year - j]
                self.output = self.out 
        if self.output[0] != '':
            return self.output
    
    def getymonth(self):
        # find all dates with yyyy-./:mm format
        self.ymonth = [w for w in self.word if re.search('^\d{4}?[-/.//://\/]\d{2}$', w)]
        for i in self.ymonth:
            for j in ["-", ":", "/", "."]:
                if j in i:
                    self.year = ""
                    self.month = ""
                    self.year = i.split(str(j))[0]
                    self.month = i.split(str(j))[1]
                if self.year in range (1900, 2019) and self.month in range (1, 13):
                    self.output[0] = self.month + "/" + self.year
        if self.output[0] != '':
            return self.output
        else:
            return None
    
    def getyday(self):
        self.yday = [w for w in self.word if re.search('^\d{4}?[-/.//://\/]\d{2}?[/.//:/-/\/]\d{2}$', w)]
        for i in self.yday:
            for j in ["-", ":", "/", "."]:
                if j in i:
                    self.year = ""
                    self.month = ""
                    self.day = ""
                    self.year = i.split(str(j))[0]
                    self.month = i.split(str(j))[1]
                    self.day = i.split(str(j))[2]
                if self.year in range (1900, 2019) and self.month in range (1, 13) and slef.day in range (1, 32):
                    self.output[0] = self.day + "/" + self.month + "/" + self.year
        if self.output[0] != '':
            return self.output
        else:
            return None  
        

In [222]:
class Search:
    def __init__(self, trans, resource):
        self.trans = trans
        self.resource = resource
        
    def con(self):
        # change year format to mm/dd/year
        if type(self.trans) == list:
            with open("results.text", "a") as outfile:
                outfile.write(self.resource)
                outfile.write("\n")
                for i in self.trans:
                    self.date = []
                    self.month = ""
                    self.day = ""
                    self.date.append(i["year"])
                    months = [{"month": 1, "mapping": ["Jan", "jan", "January", "january"]},
                                 {"month": 2, "mapping": ["Feb", "feb", "February", "february"]},
                                 {"month": 3, "mapping": ["Mar", "mar", "March", "march"]},
                                 {"month": 4, "mapping": ["Apr", "apr", "April", "april"]},
                                 {"month": 5, "mapping": ["May", "may", "05"]},
                                 {"month": 6, "mapping": ["Jun", "jun", "June", "june"]},
                                 {"month": 7, "mapping": ["Jul", "jul", "July", "july"]},
                                 {"month": 8, "mapping": ["Aug", "aug", "August", "august"]},
                                 {"month": 9, "mapping": ["Sep", "sep", "September", "september"]},
                                 {"month": 10, "mapping": ["Oct", "oct", "October", "october"]},
                                 {"month": 11, "mapping": ["Nov", "nov", "November", "november"]},
                                 {"month": 12, "mapping": ["Dec", "dec", "December", "december"]}]
                    days = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31"]
                    for j in range(-5, 6):
                        # look through the grams for month
                        if j == 0:
                            continue   
                        for x in months:
                            self.month = i["grams"][str(j) + "-gram"].replace(".", "")
                            if self.month in x["mapping"]:
                                self.date[0] = (str(x["month"]) + "/" + i["year"])
                                # look for a day in the next gram
                                if j+1 != 0 and j+1<6:
                                    if i["grams"][str(j+1) + "-gram"] in days: 
                                        self.day = i["grams"][str(j+1) + "-gram"]                      
                                        self.date[0] = (str(x["month"]) + "/" +self.day + "/" + i["year"])
                    print (self.date)
                    print ("da")
                    outfile.writelines(self.date)
                    outfile.write("\n")
                            
            #print (trans)
            #print (type(trans))
       # ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec", "March", "April", "June", "July", "Augest", "September", "October", "November", "December"]

In [224]:
dates = Query(1, 1)
extracts = dates.isVersionOf()

http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/76/53/71/84/765371841
['2006']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/76/53/72/62/765372626
['2001']
da
['2001']
da
['']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/76/53/72/13/765372138
['2012']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/cc/53/4f/p1/cc534fp14s
['2010']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/mk/61/rh/31/mk61rh31f
['2012']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/b2/77/3z/56/b2773z56b
['2014']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/2f/75/r9/14/2f75r914c
['2010']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/b2/77/3v/90/b2773v901
['2011']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/76/53/72/00/765372006
['2012']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/2f/75/r9/66/2f75r9664
['2012']
da
http://gillingham.library.ualberta.ca:8

da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/ch/98/9r/34/ch989r348v
['2015']
da
['2015']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/c5/99/9n/33/c5999n339q
['2015']
da
['2015']
da
['2015']
da
['2016']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/c8/33/6h/20/c8336h204h
['2016']
da
['2016']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/c5/13/8j/f0/c5138jf02p
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/2n/49/t2/71/2n49t271j
['2012']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/w9/50/51/01/w9505101c
['2012']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/c9/s1/61/64/c9s1616409
['2013']
da
['2013']
da
['2014']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/c9/z9/03/00/c9z903003p
['2016']
da
['5/16/2016']
da
['2016']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/76/53/71/69/76537169x
['2011']
da
['1936']
da
['2006']
da
['2011']
da
http://g

http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/jw/82/7c/63/jw827c636
['2010']
da
['2013']
da
['2013']
da
['2013']
da
['2013']
da
['2013']
da
['9/4/2013']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/cw/08/92/b0/cw0892b06h
['']
da
['2012']
da
['']
da
['2012']
da
['2014']
da
['']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/c5/34/fq/28/c534fq28v
['2012']
da
['2012']
da
['2011']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/c5/34/fq/34/c534fq340
['1984']
da
['11/1974']
da
['1975']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/c5/t3/4s/j5/c5t34sj58s
['2014']
da
['2013']
da
['2013']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/3n/20/40/01/3n204001k
['1984']
da
['11/1974']
da
['1975']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/bg/25/7f/84/bg257f844
['2014']
da
['12/2014']
da
['2014']
da
['2014']
da
['2014']
da
['']
da
['']
da
http://gillingham.library.ualberta.ca:8080/fedo

['2012']
da
['2012']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/mp/48/sd/19/mp48sd19p
['2012']
da
['2009']
da
['2009']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/cc/08/hg/44/cc08hg44s
['2012']
da
['2012']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/db/78/tc/55/db78tc55x
['2009']
da
['2010']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/c1/j9/2g/77/c1j92g778p
['2016']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/kw/52/j9/04/kw52j904w
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/73/66/65/35/736665352
['2014']
da
['2014']
da
['2014']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/tx/31/qm/44/tx31qm44j
['2015']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/1j/92/g8/31/1j92g831d
['12/2009']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/b2/77/3v/94/b2773v944
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/2v/23/vx/44/2v23vx

['2008']
da
['9/2008']
da
['2009']
da
['11/2009']
da
['11/2009']
da
['2009']
da
['11/2009']
da
['11/2009']
da
['2009']
da
['11/2009']
da
['11/2009']
da
['2012']
da
['12/2012']
da
['2012']
da
['10/3/2012']
da
['2013']
da
['2013']
da
['2014']
da
['12/2014']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/cr/78/1w/g2/cr781wg262
['2015']
da
['2015']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/cc/f9/5j/b7/ccf95jb707
['2013']
da
['2016']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/86/23/hz/06/8623hz06b
['2010']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/qj/72/p8/37/qj72p837k
['2013']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/70/79/57/95/707957952
['3/2011']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/z8/90/rt/35/z890rt359
['2011']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/c7/m0/1b/k9/c7m01bk93g
['2016']
da
['2015']
da
['2014']
da
http://gillingham.library.ualb

...

['2011']
da
['2007']
da
['']
da
['2011']
da
['2010']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/hm/50/tt/27/hm50tt27s
['2011']
da
['2012']
da
['12/2012']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/b2/77/3w/04/b2773w045
['2012']
da
['2011']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/cv/d6/6w/00/cvd66w003h
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/xs/55/mc/99/xs55mc99r
['2011']
da
['2011']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/cw/66/34/38/cw6634386r
['2013']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/ck/64/3b/13/ck643b135x
['2016']
da
['2014']
da
['2011']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/bn/k3/22/d3/bnk322d34j
['2011']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/rv/04/2t/29/rv042t299
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/c0/86/12/n6/c08612n671
['2014']
da
http://gillingham.library.ualberta.ca:8080

['']
da
['4/2011']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/0z/70/8w/86/0z708w860
['2012']
da
['2011']
da
['2010']
da
['2009']
da
['2009']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/3n/20/3z/53/3n203z53z
['2010']
da
['2011']
da
['2011']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/c2/j6/2s/50/c2j62s5037
['2013']
da
['2010']
da
['2011']
da
['2012']
da
['2012']
da
['2012']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/c2/z1/0w/q3/c2z10wq35c
['2012']
da
['2010']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/c2/n4/9t/19/c2n49t192w
['2014']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/sb/39/79/14/sb397914f
['2013']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/1r/66/j3/45/1r66j345q
['10/2012']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/1c/18/dg/59/1c18dg59n
['2012']
da
['2013']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/dv/1

http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/cv/m4/0x/r7/cvm40xr794
['2015']
da
['2016']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/xs/55/md/38/xs55md387
['2012']
da
['2012']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/b5/64/4s/00/b5644s009
['2008']
da
['2009']
da
['2010']
da
['']
da
['2012']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/cw/95/05/06/cw95050623
['12/23/2014']
da
['11/28/2013']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/cb/v7/3c/06/cbv73c066x
['2010']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/cq/r4/6r/08/cqr46r084p
['2013']
da
['2015']
da
['2015']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/zk/51/vj/04/zk51vj04h
['2011']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/2v/23/vv/06/2v23vv06j
['2012']
da
['2011']
da
['2011']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/f7/62/3d/25/f7623d25p
['2011']
da
['2011']
d

da
['2012']
da
['2012']
da
['']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/8w/32/r6/87/8w32r687j
['10/2011']
da
['4/2012']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/8s/45/qc/61/8s45qc61k
['2013']
da
['2014']
da
['2015']
da
['2014']
da
['2015']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/cc/z3/0p/s8/ccz30ps87h
['2013']
da
['2014']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/h9/89/r4/54/h989r4544
['2012']
da
['2012']
da
['2008']
da
['2011']
da
['2013']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/tt/44/pq/48/tt44pq48d
['2013']
da
['2014']
da
['2009']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/9g/54/xj/45/9g54xj45n
['2012']
da
['2014']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/4m/90/dv/73/4m90dv732
['2012']
da
['2010']
da
['2011']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/j9/60/23/28/j9602328f
http://gillingham.library.ualberta.ca:8

['2015']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/69/69/z2/30/6969z230n
['2013']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/1z/40/ks/98/1z40ks98f
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/22/27/mr/04/2227mr044
['2011']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/22/27/mr/28/2227mr28q
['2010']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/bd/79/1s/g1/bd791sg19h
['9/30/2011']
da
['9/30/2011']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/mp/48/sc/97/mp48sc97v
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/2b/88/qg/15/2b88qg15t
['2015']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/n8/70/zt/77/n870zt77b
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/90/19/s3/27/9019s327n
['2013']
da
http://gillingham.library.ualberta.ca:8080/fedora/rest/prod/90/19/s5/06/9019s5069
['2014']
da
['2015']
da
http://gillingham.library.ualberta.ca:8080/fedora

In [ ]:
sparql = SPARQLWrapper(sparql_mig_dev)
sparql.setMethod("POST")
sparql.setReturnFormat(JSON)
for i in [";", ":", "-", "/", ".", ",", " "]:
    query = "prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT (count(?s) as ?count) WHERE {?s dcterm:created ?value. filter(contains(str(?value),  str('%s')))}" % (i)
    sparql.setQuery(query)
    r  = sparql.query().convert()['results']['bindings']
    for triple in r:
        if triple['count']['value'] != '':
            print (triple['count']['value'])

In [ ]:
sparql = SPARQLWrapper(sparql_mig_dev)
sparql.setMethod("POST")
sparql.setReturnFormat(JSON)
query = "prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT ?s ?value WHERE {?s dcterm:isVersionOf ?value. {select ?s where {?s dcterm:created ?v filter(?v='')}} filter(?value != '')}"
sparql.setQuery(query)
r  = sparql.query().convert()['results']['bindings']
with open('triple.csv', 'w+') as p:
    for triple in r:
        p.write(triple['s']['value'] + " \t " + triple['value']['value'] + "\n")